# Imports

In [39]:
import logging as log
from _decimal import Decimal
from multiprocessing import Queue
from threading import Thread
from time import sleep
from typing import Union, List, Optional, Dict, Type, Any, Callable

from eth_typing import ChecksumAddress
from web3.types import EventData

from rubi.contracts import (
    RubiconMarket,
    RubiconRouter,
    ERC20,
)
from rubi.network import (
    Network,
)
from rubi.types import (
    OrderSide,
    NewMarketOrder,
    NewLimitOrder,
    Pair,
    OrderBook,
    PairDoesNotExistException,
    BaseEvent,
    OrderEvent,
    Transaction,
    BaseNewOrder,
    NewCancelOrder,
    UpdateLimitOrder
)

import os
import time
import json
import pytest
import logging as log
from eth_utils import to_wei
from eth_tester import PyEVMBackend
from web3 import EthereumTesterProvider, Web3
from dotenv import load_dotenv
from rubi import Client

# Creating instances and fixtures

In [40]:
load_dotenv(".env")
http_node_url = os.getenv("HTTP_NODE_URL")
wallet = os.getenv("DEV_WALLET")
key = os.getenv("DEV_KEY")

In [55]:
# set a fixture to return a tester provider intance 
def tester_provider():
    eth_tester_provider = EthereumTesterProvider()
    eth_tester_provider.ethereum_tester.backend = PyEVMBackend.from_mnemonic(
        'test test test test test test test test test test test junk',
        genesis_state_overrides={'balance': to_wei(1000000, 'ether')}
    )
    # return EthereumTesterProvider()
    return eth_tester_provider

# set a fixture to return the eth_tester object from the tester provider instance
def eth_tester(tester_provider):
    return tester_provider.ethereum_tester

# set a fixture to return a web3 instance instantiated from the tester provider
def w3(tester_provider):
    return Web3(tester_provider)

# a function to add an account to the eth_tester object given the private key
def add_account(eth_tester):
    new = eth_tester.add_account('0x58d23b55bc9cdce1f18c2500f40ff4ab7245df9a89505e9b1fa4851f623d241d')
    return {'address' : new, 'key': '0x58d23b55bc9cdce1f18c2500f40ff4ab7245df9a89505e9b1fa4851f623d241d'}

# a function to add another account to the eth_tester object given the private key
def add_account_buyer(eth_tester):
    new = eth_tester.add_account('0x58d23b55bc9cdce1f18c2500f40ff4ab7245df9a89505e9b1fa4851f623d2420')
    return {'address' : new, 'key': '0x58d23b55bc9cdce1f18c2500f40ff4ab7245df9a89505e9b1fa4851f623d2420'}

In [80]:
# set a fixture to return a RubiconMarket.sol instance
def market_contract(eth_tester, w3):
    # set the test addresses
    deploy_address = eth_tester.get_accounts()[0]
    fee_address = eth_tester.get_accounts()[1]

    # load the contract abi and bytecode
    path = "/Users/ishandhanani/Desktop/repos/rbkn/rubi-py/rubi/tests/abis/RubiconMarket.json"

    with open(path, 'r') as f:
        contract_interface = json.load(f)
    f.close()
    abi = contract_interface["abi"]
    bytecode = contract_interface["bytecode"]

    # set up the contract instance 
    market_deployement = w3.eth.contract(abi=abi, bytecode=bytecode)

    # create a transaction to deploy the contract
    txn = market_deployement.constructor().transact({'from': deploy_address})

    # wait for the transaction to be mined and return an instance of the contract
    receipt = w3.eth.wait_for_transaction_receipt(txn, 180)

    # now initialize the contract with the test data
    market = w3.eth.contract(address=receipt.contractAddress, abi=abi)
    init_txn = market.functions.initialize(True, fee_address).transact()

    try:
        w3.eth.wait_for_transaction_receipt(init_txn, 180)
    except Exception as e:
        log.warning('market contract failed to initialize: ', e)

    return market

# set a fixture to initialize a dictionary of erc20 contracts
def erc20s(market_contract, add_account, add_account_buyer, eth_tester, w3):

    # set the test addresses
    deploy_address = eth_tester.get_accounts()[0]
    user_one = eth_tester.get_accounts()[1]
    user_two = eth_tester.get_accounts()[2]
    user_new = add_account['address']
    user_buyer = add_account_buyer['address']

    # load the contract abi and bytecode
    path = "/Users/ishandhanani/Desktop/repos/rbkn/rubi-py/rubi/tests/abis/ERC20MockDecimals.json"
    #path = f"{os.path.dirname(os.path.realpath(__file__))}/abis/ERC20MockDecimals.json"
    with open(path, 'r') as f:
        contract_interface = json.load(f)
    f.close()
    abi = contract_interface["abi"]
    bytecode = contract_interface["bytecode"]

    # set up the contract instance 
    erc20_deployement = w3.eth.contract(abi=abi, bytecode=bytecode)

    # set the constructor arguments -> cow_setup = name, symbol, supply, decimals
    supply = 420 * 10**18

    # deploy the contract
    cow_deploy = erc20_deployement.constructor("defi cowboy", "COW", supply, 18).transact({'from': deploy_address})
    eth_deploy = erc20_deployement.constructor("ether", "ETH", supply, 18).transact({'from': deploy_address})
    blz_deploy = erc20_deployement.constructor("blaze it", "BLZ", supply, 18).transact({'from': deploy_address})

    # get the contract address
    cow_receipt = w3.eth.wait_for_transaction_receipt(cow_deploy, 180)
    eth_receipt = w3.eth.wait_for_transaction_receipt(eth_deploy, 180)
    blz_receipt = w3.eth.wait_for_transaction_receipt(blz_deploy, 180)

    # connect to the contract
    try:
        cow = w3.eth.contract(address=cow_receipt.contractAddress, abi=abi)
        eth = w3.eth.contract(address=eth_receipt.contractAddress, abi=abi)
        blz = w3.eth.contract(address=blz_receipt.contractAddress, abi=abi)
    except Exception as e:
        log.warning('there was an error connecting to the erc20 contracts: ', e)

    # send 100 tokens & 100 eth from the deployer to user_new and user_buyer
    w3.eth.send_transaction({'from': deploy_address, 'to': user_new, 'value': 100 * 10**18})
    w3.eth.send_transaction({'from': deploy_address, 'to': user_buyer, 'value': 100 * 10**18})
    cow.functions.transfer(user_new, 100 * 10**18).transact({'from': deploy_address})
    eth.functions.transfer(user_new, 100 * 10**18).transact({'from': deploy_address})
    blz.functions.transfer(user_new, 100 * 10**18).transact({'from': deploy_address})
    cow.functions.transfer(user_buyer, 100 * 10**18).transact({'from': deploy_address})
    eth.functions.transfer(user_buyer, 100 * 10**18).transact({'from': deploy_address})
    blz.functions.transfer(user_buyer, 100 * 10**18).transact({'from': deploy_address})

    # set the max approval for the erc20s
    max_approval = 2**256 - 1

    # approve the market contract to spend the strategist's tokens
    cow.functions.approve(market_contract.address, max_approval).transact({'from': deploy_address})
    cow.functions.approve(market_contract.address, max_approval).transact({'from': user_one})
    cow.functions.approve(market_contract.address, max_approval).transact({'from': user_two})
    cow.functions.approve(market_contract.address, max_approval).transact({'from': user_new})
    cow.functions.approve(market_contract.address, max_approval).transact({'from': user_buyer})
    eth.functions.approve(market_contract.address, max_approval).transact({'from': deploy_address})
    eth.functions.approve(market_contract.address, max_approval).transact({'from': user_one})
    eth.functions.approve(market_contract.address, max_approval).transact({'from': user_two})
    eth.functions.approve(market_contract.address, max_approval).transact({'from': user_new})
    eth.functions.approve(market_contract.address, max_approval).transact({'from': user_buyer})
    blz.functions.approve(market_contract.address, max_approval).transact({'from': deploy_address})
    blz.functions.approve(market_contract.address, max_approval).transact({'from': user_one})
    blz.functions.approve(market_contract.address, max_approval).transact({'from': user_two})
    blz.functions.approve(market_contract.address, max_approval).transact({'from': user_new})
    blz.functions.approve(market_contract.address, max_approval).transact({'from': user_buyer})

    erc20 = {'cow': cow, 'eth': eth, 'blz': blz}
    print("done")

    return erc20

# set a fixture to return a RubiconRouter.sol instance
def router_contract(market_contract, erc20s, eth_tester, w3):

    # set the test addresses
    deploy_address = eth_tester.get_accounts()[0]

    # load the contract abi and bytecode
    path = "/Users/ishandhanani/Desktop/repos/rbkn/rubi-py/rubi/tests/abis/RubiconRouter.json"
    with open(path, 'r') as f:
        contract_interface = json.load(f)
    f.close()
    abi = contract_interface["abi"]
    bytecode = contract_interface["bytecode"]

    # set up the contract instance 
    router_deployement = w3.eth.contract(abi=abi, bytecode=bytecode)

    # create a transaction to deploy the contract
    txn = router_deployement.constructor().transact({'from': deploy_address})

    # wait for the transaction to be mined and return an instance of the contract
    receipt = w3.eth.wait_for_transaction_receipt(txn, 180)

    # now initialie the contract with the test data
    router = w3.eth.contract(address=receipt.contractAddress, abi=abi)
    init_txn = router.functions.startErUp(market_contract.address, erc20s['cow'].address).transact()

    try:
        w3.eth.wait_for_transaction_receipt(init_txn, 180)
    except Exception as e:
        log.warning('router contract failed to initialize:', e)

    # return {'router' : router, 'market' : market_contract}
    return router

# set a fixture to return a MarketAidFactory.sol instance
def factory_contract(market_contract, eth_tester, w3):

    # set the test addresses
    deploy_address = eth_tester.get_accounts()[0]
    
    # load the contract abi and bytecode
    #path = f"{os.path.dirname(os.path.realpath(__file__))}/abis/MarketAidFactory.json"
    path = "/Users/ishandhanani/Desktop/repos/rbkn/rubi-py/rubi/tests/abis/MarketAidFactory.json"
    with open(path, 'r') as f:
        contract_interface = json.load(f)
    f.close()
    abi = contract_interface["abi"]
    bytecode = contract_interface["bytecode"]

    # set up the contract instance
    factory_deployement = w3.eth.contract(abi=abi, bytecode=bytecode)

    # create a transaction to deploy the contract
    txn = factory_deployement.constructor().transact({'from': deploy_address})

    # wait for the transaction to be mined and return an instance of the contract
    receipt = w3.eth.wait_for_transaction_receipt(txn, 180)

    # now initialize the contract with the test data
    factory = w3.eth.contract(address=receipt.contractAddress, abi=abi)
    init_txn = factory.functions.initialize(market_contract.address).transact()

    try:
        w3.eth.wait_for_transaction_receipt(init_txn, 180)
    except Exception as e:
        log.warning('factory contract failed to initialize: ', e)
    
    return factory

# set a fixture to return a MarketAide.sol instance
def aid_contract(factory_contract, eth_tester, w3):

    # set the test addresses
    strategist_address = eth_tester.get_accounts()[2]

    # load the contract abi and bytecode
    path = "/Users/ishandhanani/Desktop/repos/rbkn/rubi-py/rubi/tests/abis/MarketAid.json"
    #path = f"{os.path.dirname(os.path.realpath(__file__))}/abis/MarketAid.json"
    with open(path, 'r') as f:
        contract_interface = json.load(f)
    f.close()
    abi = contract_interface["abi"]

    # create a marketaid instance from the factory contract
    create_txn = factory_contract.functions.createMarketAidInstance().transact({'from': strategist_address})

    # wait for the transaction to be mined 
    try: 
        w3.eth.wait_for_transaction_receipt(create_txn, 180)
    except Exception as e:
        print('failed to create a market aid instance: ', e)

    # get the address of the new market aid instance
    first_aid = factory_contract.functions.getUserMarketAids(strategist_address).call()[0]

    # TODO: see if you simply can return the receipt and then use the receipt to get the contract address
    # wait for the transaction to be mined and return an instance of the contract
    # receipt = w3.eth.wait_for_transaction_receipt(create_txn, 180)

    # now initialie the contract with the test data
    aide = w3.eth.contract(address=first_aid, abi=abi)
    
    return aide

### deploy the contracts

## init the fixtures

In [81]:
testerProv = tester_provider()
ethTester = testerProv.ethereum_tester # do not need that other fixture
w3Instance = w3(testerProv)
addAcct = add_account(ethTester)
addAcctBuyer = add_account_buyer(ethTester)
market = market_contract(ethTester, w3Instance)
erc20dict = erc20s(market, addAcct, addAcctBuyer, ethTester, w3Instance)
router = router_contract(market, erc20dict, ethTester, w3Instance)
factory = factory_contract(market, ethTester, w3Instance)
aid = aid_contract(factory, ethTester, w3Instance)

done


## init network

In [ ]:
ta={
    "WETH":"0xE412a307764cCBE02E055e926516ebD74230cfE0",
    "USDC":"0xcC5f8571D858DAD7fA2238FB9df4Ad384493013C",
    "TEST":"0x6aeda41c98ab5399044fc36162B57d39c13b658a",
    "USDT":"0x97fcdaA045F4a3Bc8Fc7ad721EFf38d66B234C7F",
    "DAI":"0xAb647DF8262580c1caB61Eb165B22616365d3C67",
    "WBTC":"0x639C6472C45037F7aA868729dFfF69eB3843f1A0"
}

rbkn = {
    "market":"0x10418D9e730fa659b0Baf0b640ee41FcF4EA2aaE",
    "router":"0xbA81dF0251A017C2fB687e5469a897529442f008"
}

Network(
    path="rubi/network_config/optimism_goerli/network.yaml",
    w3=w3Instance,
    name= "Polygon Mumbai",
    chain_id=80001,
    currency="MATIC",
    rpc_url="https://rpc-mumbai.maticvigil.com/",
    explorer_url= "https://mumbai.polygonscan.com/",
    rubicon= rbkn,
    token_addresses=ta
)

In [37]:
def test_network_instance():

    # Create a Network instance using the from_config method
    network = Network.from_config(http_node_url=http_node_url)

    # Assert the attributes of the Network instance
    print(network.w3)
    print(Web3(Web3.HTTPProvider(http_node_url)))
    assert isinstance(network.w3, Web3(Web3))
    assert network.w3 == Web3.HTTPProvider(http_node_url)
    assert network.name == "Polygon Mumbai"  # Adjust the expected value based on your network configuration
    assert network.chain_id == 80001  # Adjust the expected value based on your network configuration
    assert network.currency == "MATIC"  # Adjust the expected value based on your network configuration
    #assert network.rpc_url == "https://example.com/rpc"  # Adjust the expected value based on your network configuration
    #assert network.explorer_url == "https://example.com/explorer"  # Adjust the expected value based on your network configuration

    # Assert the attributes of the RubiconContracts instance within the Network instance
    #assert isinstance(network.rubicon, RubiconContracts)
    #assert isinstance(network.rubicon.market, ContractRepr)
    #assert isinstance(network.rubicon.router, ContractRepr)


In [82]:
ntwrk = Network.from_config(http_node_url=http_node_url)

In [84]:
client_init = Client(
    network=ntwrk,
    wallet=wallet,
    key=key
)

In [52]:
client_init_nokeywallet = Client(
    network=ntwrk
)

In [48]:
client_http = Client.from_http_node_url(
            http_node_url=http_node_url,
            wallet=wallet,
            key=key
        )

In [53]:
# Test client creation
assert isinstance(client_http, Client)
assert isinstance(client_init, Client)
# Test if the wallet attribute is set correctly when a valid wallet address is provided.
assert client_init.wallet == wallet
assert client_http.wallet == wallet
assert isinstance(client_init.wallet, str)
assert isinstance(client_http.wallet, str)
# Test that no key/wallet exists when not init
assert client_init_nokeywallet.key is None
assert client_init_nokeywallet.wallet is None
# Test if the key attribute is set correctly when a key is provided.
assert client_init.key == key
assert client_http.key == key
assert isinstance(client_init.key, str)
assert isinstance(client_http.key, str)
# Test if the market/router have correct types and are init
assert isinstance(client_init.market, RubiconMarket)
assert isinstance(client_http.market, RubiconMarket)
assert isinstance(client_init.router, RubiconRouter)
assert isinstance(client_http.router, RubiconRouter)
assert isinstance(client_http.key, str)
# Test if the _pairs attribute is initialized as an empty dictionary.
assert len(client_http._pairs.keys()) == 0
assert len(client_init._pairs.keys()) == 0
# Test if the message_queue attribute is set to None when no queue is provided.
assert client_init.message_queue is None
assert client_http.message_queue is None

## test pair methods

In [54]:
client_http.add_pair(
    pair_name="WETH/USDC", 
    base_asset_allowance=Decimal("1"), 
    quote_asset_allowance=Decimal("2000")
)

ExtraDataLengthError: The field extraData is 97 bytes, but should be 32. It is quite likely that you are connected to a POA chain. Refer to http://web3py.readthedocs.io/en/stable/middleware.html#geth-style-proof-of-authority for more details. The full extraData is: HexBytes('0xd682030a83626f7288676f312e31392e37856c696e7578000000000000000000652b7c35c8287a124cb1f06306eeef10d80a2a721f765dcc1b63c464a90fd3833e60946d421730c765b170a7ad1744cbc59d70ce331931558b0c39348e50dcac00')